In [1]:
import numpy as np
import os
import sys 

sys.path.insert(1, '/workspaces/baseline/railseg')
from pcd_processing import pcd_to_las

In [2]:
ignore_index = -1
learning_map = {
            0: ignore_index,  # "unlabeled"
            1: ignore_index,  # "outlier" mapped to "unlabeled" --------------------------mapped
            10: 0,  # "car"
            11: 1,  # "bicycle"
            13: 4,  # "bus" mapped to "other-vehicle" --------------------------mapped
            15: 2,  # "motorcycle"
            16: 4,  # "on-rails" mapped to "other-vehicle" ---------------------mapped
            18: 3,  # "truck"
            20: 4,  # "other-vehicle"
            30: 5,  # "person"
            31: 6,  # "bicyclist"
            32: 7,  # "motorcyclist"
            40: 8,  # "road"
            44: 9,  # "parking"
            48: 10,  # "sidewalk"
            49: 11,  # "other-ground"
            50: 12,  # "building"
            51: 13,  # "fence"
            52: ignore_index,  # "other-structure" mapped to "unlabeled" ------------------mapped
            60: 8,  # "lane-marking" to "road" ---------------------------------mapped
            70: 14,  # "vegetation"
            71: 15,  # "trunk"
            72: 16,  # "terrain"
            80: 17,  # "pole"
            81: 18,  # "traffic-sign"
            99: ignore_index,  # "other-object" to "unlabeled" ----------------------------mapped
            252: 0,  # "moving-car" to "car" ------------------------------------mapped
            253: 6,  # "moving-bicyclist" to "bicyclist" ------------------------mapped
            254: 5,  # "moving-person" to "person" ------------------------------mapped
            255: 7,  # "moving-motorcyclist" to "motorcyclist" ------------------mapped
            256: 4,  # "moving-on-rails" mapped to "other-vehicle" --------------mapped
            257: 4,  # "moving-bus" mapped to "other-vehicle" -------------------mapped
            258: 3,  # "moving-truck" to "truck" --------------------------------mapped
            259: 4,  # "moving-other"-vehicle to "other-vehicle" ----------------mapped
        }

In [15]:
example_sample = "/workspaces/baseline/data/semantic_kitti/dataset/sequences/04/velodyne/000000.bin"
data_path = example_sample

with open(data_path, "rb") as b:
    scan = np.fromfile(b, dtype=np.float32).reshape(-1, 4)
    coord = scan[:, :3]
    strength = scan[:, -1].reshape([-1, 1])

    label_file = data_path.replace("velodyne", "labels").replace(".bin", ".label")
    if os.path.exists(label_file):
        with open(label_file, "rb") as a:
            segment = np.fromfile(a, dtype=np.int32).reshape(-1)
            segment = np.vectorize(learning_map.__getitem__)(
                segment & 0xFFFF
            ).astype(np.int32)
    else:
        segment = np.zeros(scan.shape[0]).astype(np.int32)
    data_dict = dict(coord=coord, strength=strength, segment=segment)

In [16]:
data_dict["segment"][data_dict["segment"]==5]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5], dtype=int32)

In [17]:
pcd_to_las(coord=data_dict["coord"],export_path ="/workspaces/baseline/exp/temporary_export/semanticKITTIsample.las" ,gt_segm = data_dict["segment"])